In [7]:
# Importation des librairies de base
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

import wrds

In [8]:
# Connexion à la base de données WRDS
db = wrds.Connection(wrds_username='vince1209')


Loading library list...
Done
Done
